Chapter 21
Creating and Updating Columns

In [71]:
# deletes variables
%reset -f

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 10)             
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    '2020-jetbrains-python-survey.csv'

jb = pd.read_csv(url)



In [ ]:

jb.info()

In [ ]:
# create a series to look at names of columns 
cols = pd.Series(jb.columns)

In [ ]:
cols

In [ ]:
# determine if a feature( column) can have multiple values(like database) and remove those.
import collections
counter = collections.defaultdict(list)

for col in sorted(jb.columns):
    period_count = col.count('.')
    if period_count >= 2:
        part_end = 2
    else:
        part_end = 1
    parts = col.split('.')[:part_end]
    temp = '.'.join(parts)
    counter['.'.join(parts)].append(col)




In [ ]:
uniq_cols = []
for cols in counter.values():
    if len(cols) ==1:
        uniq_cols.extend(cols)

In [ ]:
uniq_cols

In [ ]:
# the age column
# note when type age the autocomple want to use aggregate use the esc key to stop this
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .age
 .value_counts(dropna=False)
 )

In [ ]:
# can't convert to Int64 because of the bug
# https://github.com/pandas-dev/pandas/issues/33254
# but you can chain it like below
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .age
 .str.slice(0,2)  # can also use str[0:2]
 .astype(float)
 #.astype('Int64')   
 )

In [ ]:
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .age
 .str.slice(0,2)  # can also use str[0:2]
 .astype(float)
 .astype('Int64')   
 )

Chaining 
https://pandas.pydata.org/docs/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
import numpy as np

(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2)
         .astype(float) .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
         .replace({'Yes': True, 'No': False, np.nan: False})
        )
    .are_you_datascientist
 )

In [ ]:
(jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2)
         .astype(float) .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
         .replace({'Yes': True, 'No': False, np.nan: False})
        )
    .company_size
    .value_counts(dropna=False)
 )

In [ ]:


company_size=lambda df_:df_.company_size.replace({'Just me': 1,
    'Not sure': np.nan, 'More than 5,000': 5000, '2-10': 2, '11-50': 11, 
    '51-500': 51, '501-1,000': 501,
    '1,001-5000': 1001})


In [ ]:
jb2 = (jb[uniq_cols]
 .rename(columns=lambda c: c.replace('.','_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2)
         .astype(float) .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
         .replace({'Yes': True, 'No': False, np.nan: False}),
         company_size=lambda df_:df_.company_size.replace({
            'Just me': 1, 'Not sure': np.nan, 
            'More than 5,000': 5000, '2–10': 2, '11–50': 11, 
            '51–500': 51, '501–1,000': 501,
            '1,001–5,000': 1001}).astype('Int64'),
            country_live=lambda df_:df_.country_live.astype('category'),
            employment_status=lambda df_:df_.employment_status
               .fillna('Other').astype('category'),
            is_python_main=lambda df_:df_.is_python_main.astype('category'),
            team_size=lambda df_:df_.team_size
               .str.split(r'-', n=1, expand=True)
               .iloc[:,0].replace('More than 40 people', 41)
               .where(df_.company_size!=1, 1).astype(float),
            years_of_coding=lambda df_:df_.years_of_coding
               .replace('Less than 1 year', .5).str.extract(r'(\d+)')
               .astype(float),
            python_years=lambda df_:df_.python_years
               .replace('Less than 1 year', .5).str.extract(r'(\d+)')
               .astype(float),
            python3_ver=lambda df_:df_.python3_version_most
               .str.replace('_', '.').str.extract(r'(\d\.\d)')
               .astype(float),
            use_python_most=lambda df_:df_.use_python_most
               .fillna('Unknow')

        )
   .drop(columns=['python2_version_most'])
    #.team_size
    #.value_counts(dropna=False)
 )

In [ ]:
jb2

In [ ]:
(jb2
 .query('team_size.isna()')
 .employment_status
 .value_counts(dropna=False)
)

In [ ]:
import catboost as cb
import numpy as np

import pandas as pd

import collections

In [ ]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols




In [ ]:
def prep_for_ml(df):
    #remove pandas types  float, object and category
    return (df
            .assign(**{col:df[col].astype(float) for col in df.select_dtypes('number')},
                    **{col:df[col].astype(str) for col in df.select_dtypes(['object', 'category'])})
    )


   

In [ ]:
def predict_col(df, col):
    #predict the missing values for team_size
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i,typ in enumerate(df.drop(columns=[col]).dtypes):
                if str(typ) == 'object':
                    cat_idx.append(i)
    X = (missing
         .drop(columns=[col])
         .values
         )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [ ]:
def tweak_jb(jb):
   uniq_cols = get_uniq_cols(jb)

   return  (jb[uniq_cols]
   .rename(columns=lambda c: c.replace('.','_'))
   .assign(age=lambda df_:df_.age.str.slice(0,2)
            .astype(float) .astype('Int64'),
            are_you_datascientist=lambda df_:df_.are_you_datascientist
            .replace({'Yes': True, 'No': False, np.nan: False}),
            company_size=lambda df_:df_.company_size.replace({
               'Just me': 1, 'Not sure': np.nan, 
               'More than 5,000': 5000, '2–10': 2, '11–50': 11, 
               '51–500': 51, '501–1,000': 501,
               '1,001–5,000': 1001}).astype('Int64'),
               country_live=lambda df_:df_.country_live.astype('category'),
               employment_status=lambda df_:df_.employment_status
                  .fillna('Other').astype('category'),
               is_python_main=lambda df_:df_.is_python_main.astype('category'),
               team_size=lambda df_:df_.team_size
                  .str.split(r'-', n=1, expand=True)
                  .iloc[:,0].replace('More than 40 people', 41)
                  .where(df_.company_size!=1, 1).astype(float),
               years_of_coding=lambda df_:df_.years_of_coding
                  .replace('Less than 1 year', .5).str.extract(r'(\d+)')
                  .astype(float),
               python_years=lambda df_:df_.python_years
                  .replace('Less than 1 year', .5).str.extract(r'(\d+)')
                  .astype(float),
               python3_ver=lambda df_:df_.python3_version_most
                  .str.replace('_', '.').str.extract(r'(\d\.\d)')
                  .astype(float),
               use_python_most=lambda df_:df_.use_python_most
                  .fillna('Unknown')

         )
      .assign(team_size=lambda df_:predict_col(df_, 'team_size')
            .astype(int))
      .drop(columns=['python2_version_most'])
      .dropna()
)

In [ ]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    '2020-jetbrains-python-survey.csv'

jb = pd.read_csv(url)



In [ ]:
pd.set_option('display.max_rows', 50)   

In [ ]:
jb2 = tweak_jb(jb)



In [ ]:
jb2